# Softmax Classification with TF

## Import Libraries

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

tf.random.set_seed(777)  # for reproducibility

2.2.0


## Softmax classifer manually with TF

In [2]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [3]:
## declare dataset
# dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
# dataset = dataset.repeat().batch(2)

nb_classes = 3 # number of classes

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [4]:
#Weight and bias setting
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335402, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418385 ],
       [ 0.3200988 ,  0.526784  , -0.7703731 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>


In [5]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[1.3657196e-02 7.9016238e-03 9.7844124e-01]
 [3.9259776e-02 1.7034741e-02 9.4370544e-01]
 [3.8038525e-01 1.6772316e-01 4.5189157e-01]
 [3.2339048e-01 5.9075940e-02 6.1753362e-01]
 [3.6299741e-06 6.2072729e-08 9.9999630e-01]
 [2.6252020e-02 1.0727962e-02 9.6301997e-01]
 [1.5652511e-05 4.2180278e-07 9.9998391e-01]
 [2.9407690e-06 3.8113328e-08 9.9999702e-01]], shape=(8, 3), dtype=float32)


In [6]:
# Softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[0.9302204  0.06200533 0.00777428]], shape=(1, 3), dtype=float32)


In [7]:
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(6.07932, shape=(), dtype=float32)


In [8]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x # x^2
dy_dx = g.gradient(y, x) # Will compute to 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [9]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.06914607, -0.6509784 ,  0.5818323 ],
       [-1.5221257 , -1.214863  ,  2.7369885 ],
       [-1.2473828 , -1.7611003 ,  3.008483  ],
       [-1.2014606 , -1.8659233 ,  3.0673838 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.15212913, -0.34219202,  0.4943211 ], dtype=float32)>]


In [10]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 2.849417
Loss at epoch 100: 0.684151
Loss at epoch 200: 0.613813
Loss at epoch 300: 0.558205
Loss at epoch 400: 0.508306
Loss at epoch 500: 0.461059
Loss at epoch 600: 0.415072
Loss at epoch 700: 0.369636
Loss at epoch 800: 0.324533
Loss at epoch 900: 0.280720
Loss at epoch 1000: 0.246752
Loss at epoch 1100: 0.232798
Loss at epoch 1200: 0.221645
Loss at epoch 1300: 0.211476
Loss at epoch 1400: 0.202164
Loss at epoch 1500: 0.193606
Loss at epoch 1600: 0.185714
Loss at epoch 1700: 0.178415
Loss at epoch 1800: 0.171645
Loss at epoch 1900: 0.165350
Loss at epoch 2000: 0.159483


### Prediction Check

In [11]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[0.00112886 0.08154664 0.91732454]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [12]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[2.1975954e-06 1.2331175e-03 9.9876475e-01]
 [1.1288590e-03 8.1546679e-02 9.1732454e-01]
 [2.2205469e-07 1.6418649e-01 8.3581328e-01]
 [6.3921457e-06 8.5045445e-01 1.4953916e-01]
 [2.6150835e-01 7.2644705e-01 1.2044546e-02]
 [1.3783254e-01 8.6214006e-01 2.7417505e-05]
 [7.4242175e-01 2.5754121e-01 3.6978530e-05]
 [9.2197561e-01 7.8023843e-02 6.0005704e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


### Convert manual softmax classifier with TF to Python Class

In [13]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)            
            return grads
    
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
            
model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 2.472669
Loss at epoch 500: 0.375229
Loss at epoch 1000: 0.229923
Loss at epoch 1500: 0.182148
Loss at epoch 2000: 0.150633


## Manual Softmax Classifier - Full Example with zoo dataset

In [14]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

tf.random.set_seed(777)  # for reproducibility

2.2.0


In [16]:
xy = np.loadtxt('../input/deeplearningforalldata/data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, -1]

nb_classes = 7  # 0 ~ 6

# Make Y data as onehot shape
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)

print(x_data.shape, Y_one_hot.shape)

(101, 16) (101, 7)


In [17]:
#Weight and bias setting
W = tf.Variable(tf.random.normal((16, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def logit_fn(X):
    return tf.matmul(X, W) + b

def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y):
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, 
                                                      from_logits=True)    
    cost = tf.reduce_mean(cost_i)    
    return cost

def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads
    
def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return accuracy

In [18]:
def fit(X, Y, epochs=1000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            # print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            acc = prediction(X, Y).numpy()
            loss = cost_fn(X, Y).numpy() 
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))

fit(x_data, Y_one_hot)

Steps: 1 Loss: 3.635028839111328, Acc: 0.1683168262243271
Steps: 100 Loss: 0.5194157958030701, Acc: 0.7920792102813721
Steps: 200 Loss: 0.31850090622901917, Acc: 0.9108911156654358
Steps: 300 Loss: 0.23534876108169556, Acc: 0.9405940771102905
Steps: 400 Loss: 0.18872135877609253, Acc: 0.9504950642585754
Steps: 500 Loss: 0.158460333943367, Acc: 0.9504950642585754
Steps: 600 Loss: 0.13703754544258118, Acc: 0.9900990128517151
Steps: 700 Loss: 0.12098979949951172, Acc: 0.9900990128517151
Steps: 800 Loss: 0.10847963392734528, Acc: 1.0
Steps: 900 Loss: 0.09843038767576218, Acc: 1.0
Steps: 1000 Loss: 0.09016557037830353, Acc: 1.0


## Softmax Classifier with tf.keras

In [ ]:
# Lab 6 Softmax Classifier
import tensorflow as tf
import numpy as np

x_raw = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_raw = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

x_data = np.array(x_raw, dtype=np.float32)
y_data = np.array(y_raw, dtype=np.float32)

nb_classes = 3

tf.model = tf.keras.Sequential()
tf.model.add(tf.keras.layers.Dense(input_dim=4, units=nb_classes, use_bias=True))  # use_bias is True, by default

# use softmax activations: softmax = exp(logits) / reduce_sum(exp(logits), dim)
tf.model.add(tf.keras.layers.Activation('softmax'))

# use loss == categorical_crossentropy
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=0.1), metrics=['accuracy'])
tf.model.summary()

history = tf.model.fit(x_data, y_data, epochs=2000)

print('--------------')
# Testing & One-hot encoding
a = tf.model.predict(np.array([[1, 11, 7, 9]]))
print(a, tf.keras.backend.eval(tf.argmax(a, axis=1)))

print('--------------')
b = tf.model.predict(np.array([[1, 3, 4, 3]]))
print(b, tf.keras.backend.eval(tf.argmax(b, axis=1)))

print('--------------')
# or use argmax embedded method, predict_classes
c = tf.model.predict(np.array([[1, 1, 0, 1]]))
c_onehot = tf.model.predict_classes(np.array([[1, 1, 0, 1]]))
print(c, c_onehot)

print('--------------')
all = tf.model.predict(np.array([[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]))
all_onehot = tf.model.predict_classes(np.array([[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]))
print(all, all_onehot)

### Softmax Classifier with tf.keras - Ful Example with Zoo Dataset

In [19]:
# Lab 6 Softmax Classifier
import tensorflow as tf
import numpy as np

# Predicting animal type based on various features
xy = np.loadtxt('../input/deeplearningforalldata/data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

'''
(101, 16) (101, 1)
'''

nb_classes = 7  # 0 ~ 6

# Convert y_data to one_hot
y_one_hot = tf.keras.utils.to_categorical(y_data, nb_classes)
print("one_hot:", y_one_hot)

tf.model = tf.keras.Sequential()
tf.model.add(tf.keras.layers.Dense(units=nb_classes, input_dim=16, activation='softmax'))
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=0.1), metrics=['accuracy'])
tf.model.summary()

history = tf.model.fit(x_data, y_one_hot, epochs=1000)

# Single data test
test_data = np.array([[0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0]]) # expected prediction == 3 (feathers)
print(tf.model.predict(test_data), tf.model.predict_classes(test_data))

# Full x_data test
pred = tf.model.predict_classes(x_data)
for p, y in zip(pred, y_data.flatten()):
    print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

(101, 16) (101, 1)
one_hot: [[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0

4/4 [==============================] - 0s 1ms/step - loss: 0.3089 - accuracy: 0.9307
Epoch 55/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.3044 - accuracy: 0.9307
Epoch 56/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.3007 - accuracy: 0.9307
Epoch 57/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.2997 - accuracy: 0.9307
Epoch 58/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.2979 - accuracy: 0.9307
Epoch 59/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.2946 - accuracy: 0.9307
Epoch 60/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.2957 - accuracy: 0.9307
Epoch 61/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.2873 - accuracy: 0.9406
Epoch 62/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.2824 - accuracy: 0.9406
Epoch 63/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.2798 - accuracy: 0.9406
Epoch 64/1000
4/4 [=====

4/4 [==============================] - 0s 1ms/step - loss: 0.1662 - accuracy: 0.9604
Epoch 137/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1644 - accuracy: 0.9604
Epoch 138/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1651 - accuracy: 0.9604
Epoch 139/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1641 - accuracy: 0.9604
Epoch 140/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1616 - accuracy: 0.9604
Epoch 141/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1615 - accuracy: 0.9604
Epoch 142/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1608 - accuracy: 0.9604
Epoch 143/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1601 - accuracy: 0.9802
Epoch 144/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1601 - accuracy: 0.9604
Epoch 145/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1580 - accuracy: 0.9802
Epoch 146/1000


4/4 [==============================] - 0s 1ms/step - loss: 0.1147 - accuracy: 1.0000
Epoch 219/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1138 - accuracy: 1.0000
Epoch 220/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1140 - accuracy: 1.0000
Epoch 221/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1173 - accuracy: 1.0000
Epoch 222/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1163 - accuracy: 1.0000
Epoch 223/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1140 - accuracy: 1.0000
Epoch 224/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1126 - accuracy: 1.0000
Epoch 225/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1119 - accuracy: 1.0000
Epoch 226/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1124 - accuracy: 1.0000
Epoch 227/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1115 - accuracy: 0.9901
Epoch 228/1000


4/4 [==============================] - 0s 1ms/step - loss: 0.0879 - accuracy: 1.0000
Epoch 301/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0881 - accuracy: 1.0000
Epoch 302/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0880 - accuracy: 1.0000
Epoch 303/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0875 - accuracy: 1.0000
Epoch 304/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0882 - accuracy: 1.0000
Epoch 305/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0876 - accuracy: 1.0000
Epoch 306/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0879 - accuracy: 1.0000
Epoch 307/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.0867 - accuracy: 1.0000
Epoch 308/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0862 - accuracy: 1.0000
Epoch 309/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0857 - accuracy: 1.0000
Epoch 310/1000


...

4/4 [==============================] - 0s 1ms/step - loss: 0.0454 - accuracy: 1.0000
Epoch 627/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0456 - accuracy: 1.0000
Epoch 628/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0451 - accuracy: 1.0000
Epoch 629/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0454 - accuracy: 1.0000
Epoch 630/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.0451 - accuracy: 1.0000
Epoch 631/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0450 - accuracy: 1.0000
Epoch 632/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.0448 - accuracy: 1.0000
Epoch 633/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0450 - accuracy: 1.0000
Epoch 634/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0448 - accuracy: 1.0000
Epoch 635/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.0448 - accuracy: 1.0000
Epoch 636/1000


4/4 [==============================] - 0s 1ms/step - loss: 0.0403 - accuracy: 1.0000
Epoch 709/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0404 - accuracy: 1.0000
Epoch 710/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0403 - accuracy: 1.0000
Epoch 711/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0403 - accuracy: 1.0000
Epoch 712/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0402 - accuracy: 1.0000
Epoch 713/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0402 - accuracy: 1.0000
Epoch 714/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0401 - accuracy: 1.0000
Epoch 715/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0404 - accuracy: 1.0000
Epoch 716/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0401 - accuracy: 1.0000
Epoch 717/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0401 - accuracy: 1.0000
Epoch 718/1000


4/4 [==============================] - 0s 1ms/step - loss: 0.0367 - accuracy: 1.0000
Epoch 791/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0366 - accuracy: 1.0000
Epoch 792/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0367 - accuracy: 1.0000
Epoch 793/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0365 - accuracy: 1.0000
Epoch 794/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0365 - accuracy: 1.0000
Epoch 795/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0365 - accuracy: 1.0000
Epoch 796/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0364 - accuracy: 1.0000
Epoch 797/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0364 - accuracy: 1.0000
Epoch 798/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0366 - accuracy: 1.0000
Epoch 799/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0365 - accuracy: 1.0000
Epoch 800/1000


4/4 [==============================] - 0s 1ms/step - loss: 0.0334 - accuracy: 1.0000
Epoch 873/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0332 - accuracy: 1.0000
Epoch 874/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0334 - accuracy: 1.0000
Epoch 875/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.0333 - accuracy: 1.0000
Epoch 876/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0333 - accuracy: 1.0000
Epoch 877/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0331 - accuracy: 1.0000
Epoch 878/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0332 - accuracy: 1.0000
Epoch 879/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0331 - accuracy: 1.0000
Epoch 880/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0333 - accuracy: 1.0000
Epoch 881/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0335 - accuracy: 1.0000
Epoch 882/1000


4/4 [==============================] - 0s 1ms/step - loss: 0.0307 - accuracy: 1.0000
Epoch 955/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0307 - accuracy: 1.0000
Epoch 956/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0306 - accuracy: 1.0000
Epoch 957/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0306 - accuracy: 1.0000
Epoch 958/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0305 - accuracy: 1.0000
Epoch 959/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0308 - accuracy: 1.0000
Epoch 960/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0307 - accuracy: 1.0000
Epoch 961/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.0306 - accuracy: 1.0000
Epoch 962/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0306 - accuracy: 1.0000
Epoch 963/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0305 - accuracy: 1.0000
Epoch 964/1000
